In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re
from PIL import Image
import datasets
import io

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MODELS = {
    "UI-TARS 7B DPO": "bytedance-research/UI-TARS-7B-DPO"
}

In [ ]:
def load_model(model_path):
    """Load model and processor with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")

    """Load model and processor."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token
    )
    processor = AutoProcessor.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, processor

In [ ]:
def evaluate_scienceqa(model, processor, dataset_name="derek-thomas/ScienceQA"):
    """Evaluate ScienceQA as a multiple-choice question dataset."""
    dataset = load_dataset(dataset_name, split="validation[:200]")  # Limit to 10 samples
    metric = load_metric("accuracy")

    results = []

    for idx, example in enumerate(dataset):
        question = example["question"]
        options = example["choices"]  # Multiple-choice options
        correct_answer_index = int(example["answer"])  # Correct choice index

        # Process image data if available
        image_data = example.get("image", None)
        image = None
        if image_data:
            if isinstance(image_data, bytes):
                image = Image.open(io.BytesIO(image_data)).convert("RGB")
            elif isinstance(image_data, dict) and "path" in image_data:
                image = Image.open(image_data["path"]).convert("RGB")
            elif isinstance(image_data, str):
                image = Image.open(image_data).convert("RGB")

        # Prepare input
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if image:
            inputs = processor(images=image, text=question, return_tensors="pt").to(device)
        else:
            inputs = processor(text=question, return_tensors="pt").to(device)

        # Generate model output
        outputs = model.generate(**inputs, max_new_tokens=50)
        predicted_answer = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

        # 🔹 Find the best match among the choices
        best_match_index = -1
        for idx, option in enumerate(options):
            if option.lower() in predicted_answer.lower():
                best_match_index = idx
                break

        # 🔹 Use the best match index if found, else default to 0
        final_prediction = best_match_index if best_match_index != -1 else 0

        # Store results
        results.append({
            "question": question,
            "options": options,
            "ground_truth": options[correct_answer_index],
            "predicted": options[final_prediction],
            "correct": correct_answer_index == final_prediction
        })

        # Add to metric
        metric.add(prediction=final_prediction, reference=correct_answer_index)

    # Print results
    for res in results:
        print(f"\n🔍 ScienceQA Example:")
        print(f"❓ Question: {res['question']}")
        print(f"🎯 Options: {res['options']}")
        print(f"✅ Ground Truth Answer: {res['ground_truth']}")
        print(f"🤖 Model Prediction: {res['predicted']}")
        print(f"✔️ Correct: {res['correct']}")

    return metric.compute()

In [ ]:
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, processor = load_model(model_path)

    print("Text-Image Reasoning (ScienceQA):", evaluate_scienceqa(model, processor))


Evaluating UI-TARS 7B DPO...


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for


🔍 ScienceQA Example:
❓ Question: What does the verbal irony in this text suggest?
According to Mr. Herrera's kids, his snoring is as quiet as a jackhammer.
🎯 Options: ['The snoring is loud.', 'The snoring occurs in bursts.']
✅ Ground Truth Answer: The snoring is loud.
🤖 Model Prediction: The snoring is loud.
✔️ Correct: True

🔍 ScienceQA Example:
❓ Question: Which animal's mouth is also adapted for bottom feeding?
🎯 Options: ['discus', 'armored catfish']
✅ Ground Truth Answer: armored catfish
🤖 Model Prediction: discus
✔️ Correct: False

🔍 ScienceQA Example:
❓ Question: Is this a sentence fragment?
During the construction of Mount Rushmore, approximately eight hundred million pounds of rock from the mountain to create the monument.
🎯 Options: ['no', 'yes']
✅ Ground Truth Answer: yes
🤖 Model Prediction: no
✔️ Correct: False

🔍 ScienceQA Example:
❓ Question: Which of the following could Wendy's test show?
🎯 Options: ['whether producing more insulin would help the bacteria grow faster', 